In [78]:
import numpy as np
import torch
from torch import nn, optim
import torch.optim as optim
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import vtk
from vtk import *
from vtk.util.numpy_support import vtk_to_numpy
import random
import os
import sys
import time
from tqdm import tqdm
from argparse import Namespace

In [79]:
# Parameters (simulating argparse in a Jupyter Notebook)
args = Namespace(
    n_neurons=320,
    n_layers=6,
    epochs=20,  # Required argument: Set the number of epochs
    batchsize=512,
    lr=0.00005,
    no_decay=False,
    decay_rate=0.8,
    decay_at_interval=True,
    decay_interval=15,
    file_name="",
    datapath='/content/downsampled_Gaussian.vti',  # Required: Set the path to your data
    outpath='./models/',
    exp_path='../logs/',
    modified_data_path='./data/',
    dataset_name='3d_data',  # Required: Set the dataset name
    vti_name='gmm_predicted.vti',  # Required: Name of the dataset
    vti_path='./data/'

)

print(args, end='\n\n')

# Assigning parameters to variables
LR = args.lr
BATCH_SIZE = args.batchsize
decay_rate = args.decay_rate
decay_at_equal_interval = args.decay_at_interval

decay = not args.no_decay
MAX_EPOCH = args.epochs

n_neurons = args.n_neurons
n_layers = args.n_layers + 2
decay_interval = args.decay_interval
outpath = args.outpath
exp_path = args.exp_path
datapath = args.datapath
modified_data_path = args.modified_data_path
dataset_name = args.dataset_name
vti_name = args.file_name+args.vti_name
vti_path = args.vti_path

# Displaying the final configuration
print(f"Learning Rate: {LR}", flush=True)
print(f"Batch Size: {BATCH_SIZE}", flush=True)
print(f"Decay Rate: {decay_rate}", flush=True)
print(f"Max Epochs: {MAX_EPOCH}", flush=True)
print(f"Number of Neurons per Layer: {n_neurons}", flush=True)
print(f"Number of Layers (including input/output): {n_layers}", flush=True)
print(f"Data Path: {datapath}", flush=True)
print(f"Output Path: {outpath}", flush=True)
print(f"Dataset Name: {dataset_name}", flush=True)
print(f"Vti Name: {vti_name}", flush=True)

Namespace(n_neurons=320, n_layers=6, epochs=20, batchsize=512, lr=5e-05, no_decay=False, decay_rate=0.8, decay_at_interval=True, decay_interval=15, file_name='', datapath='/content/downsampled_Gaussian.vti', outpath='./models/', exp_path='../logs/', modified_data_path='./data/', dataset_name='3d_data', vti_name='gmm_predicted.vti', vti_path='./data/')

Learning Rate: 5e-05
Batch Size: 512
Decay Rate: 0.8
Max Epochs: 20
Number of Neurons per Layer: 320
Number of Layers (including input/output): 8
Data Path: /content/downsampled_Gaussian.vti
Output Path: ./models/
Dataset Name: 3d_data
Vti Name: gmm_predicted.vti


In [80]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('Device running:', device)

Device running: cuda


In [81]:
class SineLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True, is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        # self.enable_dropout = enable_dropout
        # self.dropout_prob = dropout_prob
        self.in_features = in_features
        # if enable_dropout:
        #     if not self.is_first:
        #         self.dropout = nn.Dropout(dropout_prob)
        self.linear = nn.Linear(in_features, out_features, bias=bias)

        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features,
                                             1 / self.in_features)
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0,
                                             np.sqrt(6 / self.in_features) / self.omega_0)


    def forward(self, x):
        x = self.linear(x)
        # if self.enable_dropout:
        #     if not self.is_first:
        #         x = self.dropout(x)
        return torch.sin(self.omega_0 * x)

In [82]:
class ResidualSineLayer(nn.Module):
    def __init__(self, features, bias=True, ave_first=False, ave_second=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        # self.enable_dropout = enable_dropout
        # self.dropout_prob = dropout_prob
        self.features = features
        # if enable_dropout:
        #     self.dropout_1 = nn.Dropout(dropout_prob)
        self.linear_1 = nn.Linear(features, features, bias=bias)
        self.linear_2 = nn.Linear(features, features, bias=bias)
        self.weight_1 = .5 if ave_first else 1
        self.weight_2 = .5 if ave_second else 1

        self.init_weights()


    def init_weights(self):
        with torch.no_grad():
            self.linear_1.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                           np.sqrt(6 / self.features) / self.omega_0)
            self.linear_2.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                           np.sqrt(6 / self.features) / self.omega_0)

    def forward(self, input):
        linear_1 = self.linear_1(self.weight_1*input)
        # if self.enable_dropout:
        #     linear_1 = self.dropout_1(linear_1)
        sine_1 = torch.sin(self.omega_0 * linear_1)
        sine_2 = torch.sin(self.omega_0 * self.linear_2(sine_1))
        return self.weight_2*(input+sine_2)

In [83]:
class MyResidualSirenNet(nn.Module):
    def __init__(self, obj):
        super(MyResidualSirenNet, self).__init__()
        # self.enable_dropout = obj['enable_dropout']
        # self.dropout_prob = obj['dropout_prob']
        self.Omega_0=30
        self.n_layers = obj['n_layers']
        self.input_dim = obj['dim']
        self.output_dim = obj['total_vars']
        self.neurons_per_layer = obj['n_neurons']
        self.layers = [self.input_dim]
        for i in range(self.n_layers-1):
            self.layers.append(self.neurons_per_layer)
        self.layers.append(self.output_dim)
        self.net_layers = nn.ModuleList()
        for idx in np.arange(self.n_layers):
            layer_in = self.layers[idx]
            layer_out = self.layers[idx+1]
            ## if not the final layer
            if idx != self.n_layers-1:
                ## if first layer
                if idx==0:
                    self.net_layers.append(SineLayer(layer_in,layer_out,bias=True,is_first=idx==0))
                ## if an intermdeiate layer
                else:
                    self.net_layers.append(ResidualSineLayer(layer_in,bias=True,ave_first=idx>1,ave_second=idx==(self.n_layers-2)))
            ## if final layer
            else:
                final_linear = nn.Linear(layer_in,layer_out)
                ## initialize weights for the final layer
                with torch.no_grad():
                    final_linear.weight.uniform_(-np.sqrt(6 / (layer_in)) / self.Omega_0, np.sqrt(6 / (layer_in)) / self.Omega_0)
                self.net_layers.append(final_linear)

    def forward(self,x):
        for net_layer in self.net_layers:
            x = net_layer(x)
        return x

In [84]:
def size_of_network(n_layers, n_neurons, d_in, d_out, is_residual = True):
    # Adding input layer
    layers = [d_in]
    # layers = [3]

    # Adding hidden layers
    layers.extend([n_neurons]*n_layers)
    # layers = [3, 5, 5, 5]

    # Adding output layer
    layers.append(d_out)
    # layers = [3, 5, 5, 5, 1]

    # Number of steps
    n_layers = len(layers)-1
    # n_layers = 5 - 1 = 4

    n_params = 0

    # np.arange(4) = [0, 1, 2, 3]
    for ndx in np.arange(n_layers):

        # number of neurons in below layer
        layer_in = layers[ndx]

        # number of neurons in above layer
        layer_out = layers[ndx+1]

        # max number of neurons in both the layer
        og_layer_in = max(layer_in,layer_out)

        # if lower layer is the input layer
        # or the upper layer is the output layer
        if ndx==0 or ndx==(n_layers-1):
            # Adding weight corresponding to every neuron for every input neuron
            # Adding bias for every neuron in the upper layer
            n_params += ((layer_in+1)*layer_out)

        else:

            # If the layer is residual then proceed as follows as there will be more weights if residual layer is included
            if is_residual:
                # doubt in the following two lines
                n_params += (layer_in*og_layer_in)+og_layer_in
                n_params += (og_layer_in*layer_out)+layer_out

            # if the layer is non residual then simply add number of weights and biases as follows
            else:
                n_params += ((layer_in+1)*layer_out)
            #
        #
    #

    return n_params

In [85]:
def compute_PSNR(arrgt,arr_recon):
    diff = arrgt - arr_recon
    sqd_max_diff = (np.max(arrgt)-np.min(arrgt))**2
    snr = 10*np.log10(sqd_max_diff/np.mean(diff**2))
    return snr

In [86]:
def srs(numOfPoints, valid_pts, percentage, isMaskPresent, mask_array):

    # getting total number of sampled points
    numberOfSampledPoints = int((valid_pts/100) * percentage)

    # storing corner indices in indices variable
    indices = set()

    # As long as we don't get the required amount of sample points keep finding the random numbers
    while(len(indices) < numberOfSampledPoints):
        rp = random.randint(0, numOfPoints-1)
        if isMaskPresent and mask_array[rp] == 0:
            continue
        indices.add(rp)

    # return indices
    return indices

In [87]:
def findMultiVariatePSNR(var_name, total_vars, actual, pred):
    # print('Printing PSNR')
    tot = 0
    psnr_list = []
    for j in range(total_vars):
        psnr = compute_PSNR(actual[:,j], pred[:,j])
        psnr_list.append(psnr)
        tot += psnr
        print(var_name[j], ' PSNR:', psnr,flush=True)
    avg_psnr = tot/total_vars
    print('\nAverage psnr : ', avg_psnr,flush=True)
     #this function is calculating the psnr of final epoch (or whenever it is called) of each variable and then averaging it
     #Thus individual epochs psnr is not calculated

    return psnr_list, avg_psnr

In [88]:
def compute_rmse(actual, predicted):
    mse = np.mean((actual - predicted) ** 2)
    return np.sqrt(mse)

def denormalizeValue(total_vars, to, ref):
    to_arr = np.array(to)
    for i in range(total_vars):
        min_data = np.min(ref[:, i])
        max_data = np.max(ref[:, i])
        to_arr[:, i] = (((to[:, i] * 0.5) + 0.5) * (max_data - min_data)) + min_data
    return to_arr

In [89]:
def makeVTI(data, val, n_predictions, n_pts, total_vars, var_name, dim, isMaskPresent, mask_arr, vti_path, vti_name, normalizedVersion = False):
    nn_predictions = denormalizeValue(total_vars, n_predictions, val) if not normalizedVersion else n_predictions
    writer = vtkXMLImageDataWriter()
    writer.SetFileName(vti_path + vti_name)
    img = vtkImageData()
    img.CopyStructure(data)
    if not isMaskPresent:
        for i in range(total_vars):
            f = var_name[i]
            temp = nn_predictions[:, i]
            arr = vtkFloatArray()
            for j in range(n_pts):
                arr.InsertNextValue(temp[j])
            arr.SetName(f)
            img.GetPointData().AddArray(arr)
        # print(img)
        writer.SetInputData(img)
        writer.Write()
        print(f'Vti File written successfully at {vti_path}{vti_name}',flush=True)
    else:
        for i in range(total_vars):
            f = var_name[i]
            temp = nn_predictions[:, i]
            idx = 0
            arr = vtkFloatArray()
            for j in range(n_pts):
                if(mask_arr[j] == 1):
                    arr.InsertNextValue(temp[idx])
                    idx += 1
                else:
                    arr.InsertNextValue(0.0)
            arr.SetName('p_' + f)
            data.GetPointData().AddArray(arr)
        # print(data)
        writer.SetInputData(data)
        writer.Write()
        print(f'Vti File written successfully at {vti_path}{vti_name}')

In [90]:
def getImageData(actual_img, val, n_pts, var_name, isMaskPresent, mask_arr):
    img = vtkImageData()
    img.CopyStructure(actual_img)
    # if isMaskPresent:
    #     img.DeepCopy(actual_img)
    # img.SetDimensions(dim)
    # img.SetOrigin(actual_img.GetOrigin())
    # img.SetSpacing(actual_img.GetSpacing())
    if not isMaskPresent:
        f = var_name
        data = val
        arr = vtkFloatArray()
        for j in range(n_pts):
            arr.InsertNextValue(data[j])
        arr.SetName(f)
        img.GetPointData().SetScalars(arr)
    else:
        f = var_name
        data = val
        idx = 0
        arr = vtkFloatArray()
        for j in range(n_pts):
            if(mask_arr[j] == 1):
                arr.InsertNextValue(data[idx])
                idx += 1
            else:
                arr.InsertNextValue(0.0)
        arr.SetName(f)
        img.GetPointData().SetScalars(arr)
    return img

In [91]:
# Variable Initialization
var_name = []
total_vars = None  # Number of variables
univariate = None  # True if dataset has one variable, else False
group_size = 5000  # Group size during testing

In [92]:
n_pts = None  # Number of points in the dataset
n_dim = None  # Dimensionality of the data
dim = None  # Other dimension-specific information

print("Decay:", decay,flush=True)
print(f'Extracting variables from path: {datapath}', end="\n\n",flush=True)

# Placeholder for data
data_array = []
scalar_data = None

Decay: True
Extracting variables from path: /content/downsampled_Gaussian.vti



In [93]:
# Reading values from .vti files
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(datapath)
reader.Update()

data = reader.GetOutput()
scalar_data = data
pdata = data.GetPointData()
n_pts = data.GetNumberOfPoints()
dim = data.GetDimensions()
n_dim = len(dim)
total_arr = pdata.GetNumberOfArrays()

print("n_pts:", n_pts, "dim:", dim, "n_dim:", n_dim, "total_arr:", total_arr,flush=True)

var_name = []
data_array = []

# Extracting data from the .vti file
for i in range(total_arr):
    a_name = pdata.GetArrayName(i)

    cur_arr = pdata.GetArray(a_name)
    n_components = cur_arr.GetNumberOfComponents()

    if n_components == 1:
        var_name.append(a_name)
        data_array.append(vtk_to_numpy(cur_arr))
    else:
        component_names = [f"{a_name}_{c}" for c in ['x', 'y', 'z'][:n_components]]
        var_name.extend(component_names)
        for c in range(n_components):
            c_data = [cur_arr.GetComponent(j, c) for j in range(n_pts)]
            data_array.append(np.array(c_data))

total_vars = len(var_name)
univariate = total_vars == 1

# Prepare numpy arrays for coordinates and variable values
cord = np.zeros((n_pts, n_dim))
val = np.zeros((n_pts, total_vars))

# Store data in numpy arrays
for i in range(n_pts):
    pt = scalar_data.GetPoint(i)
    cord[i, :] = pt
    val[i, :] = [arr[i] for arr in data_array]

# Display final information
print("Total Variables:", total_vars, flush=True)
print("Univariate:", univariate, flush=True)
print("Coordinates Shape:", cord.shape, flush=True)
print("Values Shape:", val.shape, flush=True)

n_pts: 512 dim: (8, 8, 8) n_dim: 3 total_arr: 2
Total Variables: 2
Univariate: False
Coordinates Shape: (512, 3)
Values Shape: (512, 2)


In [94]:
real_data=val.copy()
# Normalize values between -1 and 1
for i in range(total_vars):
    min_data = np.min(val[:, i])
    max_data = np.max(val[:, i])
    # norm_params[var_name[i]] = (min_data, max_data)
    val[:, i] = 2.0 * ((val[:, i] - min_data) / (max_data - min_data) - 0.5)

# Normalize Coordinates to [-1,1]
for i in range(n_dim):
    # Use (dim[i]-1] so that coordinates go from 0 to dim[i]-1.
    cord[:, i] = 2.0 * (cord[:, i] / (dim[i] - 1) - 0.5)


# # Save normalized values and coordinates

n_cord = cord.copy()
n_val = val.copy()

# Convert normalized data to PyTorch tensors
torch_coords = torch.from_numpy(n_cord)
torch_vals = torch.from_numpy(n_val)

# Display dataset details
print('Dataset Name:', dataset_name, flush=True)
print('Total Variables:', total_vars, flush=True)
print('Variables Name:', var_name, end="\n\n", flush=True)
print('Total Points in Data:', n_pts, flush=True)
print('Dimension of the Dataset:', dim, flush=True)
print('Number of Dimensions:', n_dim, flush=True)
print('Coordinate Tensor Shape:', torch_coords.shape, flush=True)
print('Scalar Values Tensor Shape:', torch_vals.shape, flush=True)

print('\n###### Data setup is complete, now starting training ######\n', flush=True)


Dataset Name: 3d_data
Total Variables: 2
Variables Name: ['Mean', 'Std']

Total Points in Data: 512
Dimension of the Dataset: (8, 8, 8)
Number of Dimensions: 3
Coordinate Tensor Shape: torch.Size([512, 3])
Scalar Values Tensor Shape: torch.Size([512, 2])

###### Data setup is complete, now starting training ######



In [95]:
# Split targets into two tensors: shape (512, 1) each
torch_vals_means = torch_vals[:, 0]
torch_vals_stds= torch_vals[:,1]
print(torch_vals_means.shape)
print(torch_vals_stds.shape)

print(torch_coords.shape)

# --- 1. Set global seeds ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# For deterministic CUDA behavior (slower but reproducible)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# --- 2. Create a generator for DataLoader ---
g = torch.Generator()
g.manual_seed(SEED)

# --- 3. DataLoaders with fixed generator ---
train_dataloader_m = DataLoader(
    TensorDataset(torch_coords, torch_vals_means),
    batch_size=BATCH_SIZE, pin_memory=True, shuffle=True,
    num_workers=4, generator=g
)

train_dataloader_s = DataLoader(
    TensorDataset(torch_coords, torch_vals_stds),
    batch_size=BATCH_SIZE, pin_memory=True, shuffle=True,
    num_workers=4, generator=g
)
if not os.path.exists(outpath):
    os.makedirs(outpath)
# Create a function to train one model on one target
def train_single_target(target_id, dataloader,obj):
    model = MyResidualSirenNet(obj).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))
    criterion = nn.MSELoss()

    train_loss_list = []
    best_epoch = -1
    best_loss = 1e8

    for epoch in tqdm(range(MAX_EPOCH)):
        model.train()
        temp_loss_list = []
        start = time.time()

        for X_train, y_train in dataloader:
            X_train = X_train.type(torch.float32).to(device)
            y_train = y_train.type(torch.float32).to(device)

            optimizer.zero_grad()
            predictions = model(X_train).squeeze()
            loss = criterion(predictions, y_train.squeeze())
            loss.backward()
            optimizer.step()

            temp_loss_list.append(loss.item())

        epoch_loss = np.mean(temp_loss_list)

        # Learning rate decay
        if decay:
            if decay_at_equal_interval:
                if epoch >= decay_interval and epoch % decay_interval == 0:
                    for param_group in optimizer.param_groups:
                        param_group['lr'] *= decay_rate
            elif epoch > 0 and epoch_loss > train_loss_list[-1]:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= decay_rate

        train_loss_list.append(epoch_loss)
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            best_epoch = epoch + 1
            torch.save(
                {"epoch": best_epoch, "model_state_dict": model.state_dict()},
                os.path.join(outpath, f'best_model_{target_id}.pth')
            )

        end = time.time()
        print(f"[Target {target_id}] Epoch {epoch+1}/{MAX_EPOCH} "
              f"| Loss: {epoch_loss} || LR: {optimizer.param_groups[0]['lr']}",flush=True)

    print(f"\n[Target {target_id}] Best Epoch: {best_epoch}, Best Loss: {best_loss:.6f}\n",flush=True)
obj = {
    'total_vars': 1,
    'dim': n_dim,
    'n_neurons': n_neurons,
    'n_layers': n_layers
}

# Train two separate models
mean_id="means"
train_single_target(mean_id, train_dataloader_m,obj)
std_id="std"
train_single_target(std_id, train_dataloader_s,obj)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


torch.Size([512])
torch.Size([512])
torch.Size([512, 3])


  0%|          | 0/20 [00:00<?, ?it/s]

[Target means] Epoch 1/20 | Loss: 0.28799548745155334 || LR: 5e-05


  5%|▌         | 1/20 [00:00<00:03,  5.01it/s]

...

[Target std] Epoch 19/20 | Loss: 0.01246025413274765 || LR: 4e-05


 95%|█████████▌| 19/20 [00:05<00:00,  5.50it/s]

[Target std] Epoch 20/20 | Loss: 0.00965617410838604 || LR: 4e-05


100%|██████████| 20/20 [00:05<00:00,  3.78it/s]


[Target std] Best Epoch: 20, Best Loss: 0.009656



In [96]:


# Paths to saved best models
model_paths = [
    os.path.join(outpath, f"best_model_{mean_id}.pth"),
    os.path.join(outpath, f"best_model_{std_id}.pth"),
]

# Both models will have total_vars = 1
obj_single = {
    'total_vars': 1,
    'dim': n_dim,
    'n_neurons': n_neurons,
    'n_layers': n_layers
}

# Load models
models = []
for path in model_paths:
    m = MyResidualSirenNet(obj_single).to(device)
    checkpoint = torch.load(path, map_location=device)
    m.load_state_dict(checkpoint["model_state_dict"])
    m.eval()
    models.append(m)

# Inference: collect predictions from both models
predictions = []
with torch.no_grad():
    for m in models:
        pred_list = []
        for i in range(0, torch_coords.shape[0], group_size):
            coords = torch_coords[i:min(i + group_size, torch_coords.shape[0])].type(torch.float32).to(device)
            vals = m(coords)  # shape [group, 1]
            pred_list.append(vals.cpu())
        pred_tensor = torch.cat(pred_list, dim=0)  # [512, 1]
        predictions.append(pred_tensor)

# Combine into shape [512, 2]
n_predictions = torch.cat(predictions, dim=1)
n_predictions=np.array(n_predictions)
print(n_predictions.shape)  # torch.Size([512, 2])
# Compute PSNR
findMultiVariatePSNR(var_name, total_vars, n_val, n_predictions)

# Compute RMSE
rmse = compute_rmse(n_val, n_predictions)
print("RMSE:", rmse,flush=True)

(512, 2)
Mean  PSNR: 27.64842535121715
Std  PSNR: 25.683287704364822

Average psnr :  26.665856527790986
RMSE: 0.09402599168229316


In [97]:
# vti saving path
vti_path = args.vti_path
if not os.path.exists(vti_path):
    os.makedirs(vti_path)
# vti name
vti_name = args.vti_name
mask_arr=False
isMaskPresent=False
makeVTI(data,real_data, n_predictions, n_pts, total_vars, var_name, dim, isMaskPresent, mask_arr, vti_path, vti_name)

Vti File written successfully at ./data/gmm_predicted.vti
